In [25]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


$\newcommand{\set}[2]{\big\{#1\,\ {\large:}\ \,#2\big\}}
\newcommand{\eps}{\varepsilon}
\newcommand{\abs}[1]{\left\vert#1\right\vert}
\newcommand{\ceil}[1]{\left\lceil#1\right\rceil}
\newcommand{\floor}[1]{\left\lfloor#1\right\rfloor}
\DeclareMathOperator{\sinal}{sinal}
$
# O método da bissecção

## $ \S 1 $ Introdução

O método da bissecção é um procedimento geral para determinação de zeros de uma função  real $ f $. Seus únicos requerimentos são que a função seja contínua e que tenhamos previamente identificado um intervalo $ [a, b] $ dentro do domínio de $ f $ onde ela troca de sinal. Seus principais méritos são:
* _Confiabilidade_: ele sempre converge a um zero, satisfeitas as hipóteses acima.
* _Simplicidade_: ele é facilmente implementado e não é necessário calcular a derivada da função.
* _Robustez_: pequenos erros de arredondamento (comparados à precisão desejada) não impactam seu resultado.

Por outro lado, o método não pode ser empregado para se localizar um zero de multiplicidade par (como o da função $ x \mapsto x^2 $). Além disto, em muitos casos é possível utilizar métodos cuja convergência é ainda mais rápida. Se queremos obter um zero de uma única função, esta diferença de desempenho não é relevante, porém a situação muda se o algoritmo precisa ser executado bilhões ou trilhões de vezes.

## $ \S 2 $ Descrição do método da bissecção

Como acima, sejam $ f $ uma função real _contínua_ e $ [a, b] $ um intervalo dentro do domínio de $ f $ com
$$
\boxed{\sinal f(a) \ne \sinal f(b)}
$$
No **método da bissecção** começamos tomando $ m $ como o **ponto médio** de $ a $ e $ b $,
$$
\boxed{m = \frac{a+b}{2}}
$$
e avaliando $ f $ aí. Há três possibilidades:
* Se $ f(m)= 0 $, então $ m $ é um zero de $ f $ e podemos terminar.
* Se $ \sinal f(m)  \neq \sinal f(a) $, então $ f $ troca de sinal em $ [a, m] $.
* Se $ \sinal f(m) = \sinal f(a) $, então $ f $ troca de sinal em $ [m, b] $.

Nos dois últimos casos, podemos restringir nossa busca a um intervalo cujo comprimento é a *metade* do anterior. O teorema do valor intermediário garante que este subintervalo ainda contém um zero, portanto podemos repetir o processo. Bissecções sucessivas eventualmente nos levarão a um zero ou a um encaixotamento de um zero por um subintervalo de comprimento tão pequeno quanto desejado. Quando isto acontecer, retornamos o ponto médio deste subintervalo como estimativa para o zero.

📝 O método da bissecção também é conhecido como _busca binária_.

📝 Não é necessário _isolar_ um zero dentro do intervalo inicial, ou seja, o método funciona mesmo que haja mais de um zero dentro de $ [a, b] $.

**Exemplo 1:** Execute em seqüência as três células abaixo para ver uma animação do método da bissecção aproximando um zero da função $ f(x) = x^3 - x - 3 $ (em $ 1.6717 $, aproximadamente).

In [ ]:
def imprime_solucao(xs, ys, freq=1):
    """
    Dados dois arrays xs e ys de mesmo comprimento, imprime um
    a cada 'freq' de seus valores por linha, na forma de uma
    tabela. Os 0-ésimos e últimos valores sempre são impressos.
    """
    def imprime_cabecalho():
        """
        Imprime o cabeçalho da tabela.
        """
        print("\n|       n      ", end="")
        print("    x_n            ", end="")
        print("    f(x_n)      |")
        print("|=================================================|")
        
        
    def imprime_linha(x, y, n):
        """
        Imprime uma das linhas da tabela.
        """
        if n == 0:
            print(f"|       a", end="")
        elif n == 1:
            print(f"|       b", end="")
        else:
            print(f"|      {n - 1:02}", end="")
        print(f"    {x:15.8f}", end="")
        print(f"    {y:15.8f}   |")
    
    
    N = len(xs)
    if freq == 0:          # Se freq == 0, imprime apenas os últimos valores.
        freq = N - 1
    imprime_cabecalho()
    for n in range(0, 2):
        imprime_linha(xs[n], ys[n], n)
    for n in range(2, N, freq):
        imprime_linha(xs[n], ys[n], n)
    if n != N - 1:
        imprime_linha(xs[N], ys[N])
    print("|_________________________________________________|\n")
        
    return None

In [50]:
def animador_bisseccao(f, a, b, N=4, titulo="", pausa=0.75):
    """
    Exibe uma animação do método da bissecção aplicado a uma função.
    Entradas:
        * Uma função real contínua f.
        * As extremidades a < b do intervalo inicial.
        * O número de iterações a serem calculadas.
        * O título a ser exibido no topo do diagrama.
        * A pausa entre dois passos consecutivos da animação, em segundos.
    Saída:
        * Duas listas xs e ys contendo as estimativas para um zero
          e os valores de f nelas, respectivamente.
    Exibe:
        * Uma animação dos elementos relevantes numa janela pop-up.
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    
    def iteracao(a, b):
        """
        Aplica um passo do método da bissecção ao intervalo [a, b].
        Retorna o ponto médio e as extremidades do intervalo seguinte.
        """
        m = 0.5 * (a + b)                      # Ponto médio do intervalo.
        if np.sign(f(a)) != np.sign(f(m)):     # [a, m] contém um zero.
            return m, a, m
        else:                                  # [m, b] contém um zero.
            return m, m, b
    
    
    P = 200                                    # Número de pontos num plot.
    grossa = 2.0                               # Espessura grossa da traço.
    media = 1.75                               # Espessura média.
    fina = 1.0                                 # Espessura fina.
    dominio = np.linspace(a, b, P)             # Gera P nodos de a a b.
    xs = [a, b]                                # Armazenará as estimativas.
    for _ in range(N):                         # Preenchendo xs.
        m, a, b = iteracao(a, b)
        xs.append(m)
    ys = [f(x) for x in xs]                    # Armazena f das estimativas.
    
    # Listas contendo coordenadas x e y para plotagem das retas verticais:
    xs_vert = [np.linspace(xs[n], xs[n], P) for n in range(N + 2)]
    ys_vert = [np.linspace(0, ys[n], P) for n in range(N + 2)]
    
    # Desenhando o gráfico de f:
    plt.axhline(y=0.0, color='black', linestyle='-', linewidth=media)
    plt.xlabel('Eixo-$ x $')
    plt.ylabel('Eixo-$ y $')
    plt.title(titulo)
    plt.grid(True)
    plt.plot(dominio, f(dominio), label='$ y = f(x) $', linewidth=media)
    plt.legend()
        
    # Marcando a no eixo-x e desenhando a reta vertical por ele:
    if pausa > 0:
        plt.pause(pausa)
    plt.plot(xs[0], 0, color='black', marker="|", mew=media)
    if pausa > 0:
            plt.pause(pausa)
    plt.plot(xs_vert[0], ys_vert[0], linestyle='-', linewidth=media, label='$ a $')
    plt.plot(xs[0], 0, color='black', marker="|", mew=media)
    plt.plot(xs[0], ys[0], color='black', marker="o", mew=0.175)
    plt.legend()
    
    # Marcando b no eixo-x e desenhando a reta vertical por ele:
    if pausa > 0:
        plt.pause(pausa)
    plt.plot(xs[1], 0, color='black', marker="|", mew=media)
    if pausa > 0:
            plt.pause(pausa)
    plt.plot(xs_vert[1], ys_vert[1], linestyle='-', linewidth=media, label='$ b $')
    plt.plot(xs[1], 0, color='black', marker="|", mew=media)
    plt.plot(xs[1], ys[1], color='black', marker="o", mew=0.175)
    plt.legend()
    
    for n in range(2, N + 2):
        if pausa > 0:
            plt.pause(pausa)
        # Marcando x_n:
        plt.plot(xs[n], 0, color='black', marker="|", mew=media)
        if pausa > 0:
            plt.pause(pausa)
        # Desenhando a reta vertical por x_n e marcando (x_n, y_n):
        plt.plot(xs_vert[n], ys_vert[n], linestyle='-',
                 linewidth=media, label=f'$ x_{n - 1} = m_{n - 1} $')
        plt.plot(xs[n], ys[n], color='black', marker="o", mew=0.175)
        plt.plot(xs[n], 0, color='black', marker="|", mew=media)
    plt.legend()
    
    return xs, ys

In [52]:
a = 0           # Extremidade esquerda do intervalo inicial, onde f vale -3.
b = 2           # Extremidade direita, onde f vale 3.
N = 5           # Número de iterações desejado.
pausa = 0.75    # Intervalo de tempo entre cada passo da animação, em segundos.
f = lambda x: x**3 - x - 3    # Função à qual o método será aplicado.
# Título a ser exibido no topo do diagrama:
titulo = "Método da bissecção para $ y = x^3 - x - 3,\ a = 0,\ b = 2 $."

xs, ys = animador_bisseccao(f, a, b, N, titulo, pausa)

![Exemplo do método da bissecção](fig_2-3_exemplo_1.png "Exemplo de aplicação do método da bissecção")

In [27]:
imprime_solucao(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|       a         0.00000000        -3.00000000   |
|       b         2.00000000         3.00000000   |
|      01         1.00000000        -3.00000000   |
|      02         1.50000000        -1.12500000   |
|      03         1.75000000         0.60937500   |
|      04         1.62500000        -0.33398438   |
|      05         1.68750000         0.11791992   |
|_________________________________________________|



## $ \S 3 $ Análise do desempenho do método da bissecção

Cada passo do método da bissecção corta o comprimento do intervalo anterior pela metade. Portanto dez passos reduzem o comprimento do intervalo original por um fator de $ 2^{10} > 1\,000 $. Mais geralmente, temos o seguinte resultado.

**Teorema 3.1 (análise do método da bissecção):** _Pelo método da bissecção, o número de iterações necessário para se localizar um zero dentro do intervalo inicial $ [a, b] $ com erro menor que $ \eps > 0 $ é dado por:_
\begin{equation*}\label{E:1}
\boxed{\ceil{\lg\bigg(\frac{b-a}{\eps}\bigg)} \quad \text{onde $ \lg = \log_2 $}} \tag{1}
\end{equation*}

**Prova:** Começando com o intervalo $ [a,b] $ original (primeiro passo), sabemos que em cada iteração há um zero dentro do intervalo atual. Após o $ n $-ésimo passo, o comprimento de $ [a, b] $ terá sido reduzido pelo fator de $ 2^{n-1} $. Mas como o resultado do método é o ponto médio do intervalo atual, o que importa é a *metade* do seu comprimento, já que esta é uma cota superior para a distância do ponto médio a um zero. Logo $ n $ deve ser grande o suficiente de modo que
$$
    \frac{b-a}{2^n} < \eps, \quad \text{ou equivalentemente,} \quad 
    n > \lg \left( \frac{b-a}{\eps} \right).
$$
O menor inteiro que satisfaz esta desigualdade é o teto do valor à direita, como em \eqref{E:1}. $\  \blacksquare $

📝 Informalmente, o resultado acima implica que para cada dígito adicional de precisão, precisamos de $ \lg 10 \approx 3.32 $ iterações a mais. Este desempenho deve ser comparado com o da busca incremental, que dependia _linearmente_ de $ \frac{b-a}{\eps} $; portanto para melhorar a precisão desta por um dígito decimal, precisamos _multiplicar_ o número de iterações por $ 10 $.

## $ \S 4 $ Critérios de parada e observações

Os critérios de parada que podemos utilizar são essencialmente os mesmos em todos os métodos para localização de zeros:

1. A distância da estimativa atual a um zero é menor que um $ \eps > 0 $ pré-escolhido.
2. O módulo do valor da função na estimativa atual é menor que um $ \delta > 0 $ escolhido previamente.
3. O número de iterações excede uma cota prefixada.

Além destas, podemos também considerar versões _relativas_ de 1 e 2, por exemplo:

4. O módulo do valor da função na estimativa $ m $ atual é menor que um $ \delta > 0 $ relativamente aos valores originais $ f(a) $ e $ f(b) $, i.e.:
$$
    \frac{\abs{f(m)}}{\min\left\{\abs{f(a)}\,,\,\abs{f(b)}\right\}} < \delta.
$$

Na implementação abaixo, o procedimento termina assim que algum dos critérios 1 ou 3 for satisfeito.

⚠️ Para que possamos sequer aplicar o método da bissecção a uma função, antes é necessário ter obtido um intervalo onde ela troca de sinal. Este requerimento preliminar geralmente pode ser satisfeito com a inspeção do gráfico da função ou com uma busca incremental (conforme explicado no caderno anterior).

⚠️ Por definição, $ a $ é um **pólo** de ordem $ m $ de uma função $ f $ se ele é um zero de multiplicidade $ m $ de $ 1 / f(x) $. Se $ f $ tiver um pólo de ordem ímpar num ponto, como o da função $ x \mapsto \frac{1}{x} $ em $ x = 0 $, então o método da bissecção poderá confundi-lo com um zero. Execute a animação abaixo para entender o que acontece nestes casos.

In [54]:
a = -0.15       # Extremidade esquerda do intervalo inicial.
b = 0.18        # Extremidade direita.
N = 6           # Número de iterações desejado.
pausa = 0.75    # Intervalo de tempo entre cada passo da animação, em segundos.
f = lambda x: 1 / x   # Função à qual o método será aplicado.
# Título a ser exibido no topo do diagrama:
titulo = "Método da bissecção para $ y = 1/x,\ a = -0.15,\ b = 0.18 $;\n"\
         "confundindo um pólo de ordem ímpar com um zero."

xs, ys = animador_bisseccao(f, a, b, N, titulo, pausa)

![Pólo de ordem ímpar](fig_2-3_exemplo_2.png "Método da bissecção confundindo um pólo de ordem ímpar com um zero")

In [32]:
imprime_solucao(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|       a        -0.15000000        -6.66666667   |
|       b         0.18000000         5.55555556   |
|      01         0.01500000        66.66666667   |
|      02        -0.06750000       -14.81481481   |
|      03        -0.02625000       -38.09523810   |
|      04        -0.00562500      -177.77777778   |
|      05         0.00468750       213.33333333   |
|      06        -0.00046875     -2133.33333333   |
|_________________________________________________|



## $ \S 5 $ Implementação do método da bissecção

In [4]:
def bisseccao(f, a, b, eps, max_iter):
    """
    Usa o método da bissecção para aproximar um zero de uma função contínua.
    Entradas:
        * A função real contínua f.
        * As extremidades a < b de um intervalo onde f troca de sinal.
        * A distância máxima tolerada eps da estimativa a um zero.
        * O número máximo max_iter de iterações.
    Saída:
        * A lista xs das estimativas e a lista ys dos valores de f nelas.
    Imprime:
        * A última estimativa, o valor de f aí, o número de iterações
          e uma cota para o erro.
    """
    from numpy import sign
    

    iteracoes = 0                         # Contador do número de iterações.
    f_a = f(a)                            # Gravando o valor de f em a.
    f_b = f(b)                            # Gravando o valor de f em b.
    xs = [a, b]                           # Lista que armazenará as estimativas.
    ys = [f_a, f_b]                       # Armazenará os valores de f nos x.
    if eps <= 0:                          # Valor inválido de eps.
        raise ValueError("A tolerância deve ser positiva!")
    if f_a == 0:                          # a é um zero.
        print("a é um zero exato.")
        return a
    elif f_b == 0:                        # b é um zero.
        print("b é um zero exato.")
        return b
    elif sign(f_a) == sign(f_b):          # Erro: [a, b] não contém um zero.
        raise ValueError("A função assume valores de mesmo"
                         "sinal nas extremidades dadas!")
    
    while (b - a) >= eps and iteracoes <= max_iter:
        m = 0.5 * (a + b)                 # Ponto médio do intervalo anterior.
        f_m = f(m)                        # Gravando o valor de f em m.
        xs.append(m)
        ys.append(f_m)
        if f_m == 0:
            print("Encontrado um zero exato.")
            return m
        elif sign(f_a) != sign(f_m):      # [a, m] contém um zero.
            b = m                         # Tome o novo b como sendo m.
        else:                             # [m, b] contém um zero.
            a = m                         # Tome o novo a como sendo m.
            f_a = f_m
        iteracoes += 1
        
    print(f"Encontrado um zero aproximado:\n{m:15.9f}")
    print(f"após {iteracoes} iterações, com erro de no máximo {b - a}.")
    print(f"O valor da função neste ponto é:\n{f(m):15.9f}")
    
    return xs, ys

## $ \S 6 $ Problemas

**Problema 1:**

(a) Estime $ \sqrt[3]{2} $ com $ 5 $ dígitos decimais de precisão usando o método da bissecção. *Dica:* Considere a função $ f(x) = x^3 - 2 $.

(b) Use o Teorema 3.1 para calcular o número de iterações necessárias.

*Solução:*

**Problema 2:** Seja $ y = \tan x $ a função tangente. Recorde que 
$$
\tan\left(\frac{\pi}{4}\right) = 1 \quad \text{e} \quad \tan\left(\frac{2\pi}{3}\right) = -\sqrt{3}
$$
Aplique o método da bissecção a $ f = \tan $ no intervalo $ \left[\frac{\pi}{4}\,,\,\frac{2\pi}{3}\right] $ e explique o resultado.

*Solução:*

**Problema 3:**

(a) Encontre a menor raiz real positiva da equação
$$
x^3 - 3.45x^2 - 5.72 x + 6.31 = 0
$$
com uma precisão melhor que $ 10^{-3} $ usando o método da bissecção.

(b) Considerando o intervalo original que você utilizou, quantos passos são necessários?

*Solução:*

**Problema 4:** Obtenha uma estimativa para $ \pi $, correta até a sexta casa decimal, utilizando o método da bissecção para aproximar o primeiro zero positivo da função seno. 

*Solução:*

**Problema 5:**

(a) Mostre que a equação $ xe^x = 1 $ possui uma única raiz em $ \mathbb{R} $. *Dica:* Calcule a derivada da função $ f(x) = xe^x - 1 $.

(b) Mostre que esta raiz se encontra no intervalo $ [0.5, 1] $.

(c) Estime esta raiz com erro máximo de $ 10^{-5} $, usando o método da bissecção.

(d) Quantos passos são necessários para garantir esta precisão?

*Solução:*

**Problema 6:** Resolva o exercício anterior para a equação $ x^2 + \ln x = 0 $ ($ x > 0 $).

*Solução:*

**Problema 7:** Encontre uma raiz de cada uma das equações abaixo com erro $ < 10^{-3} $. *Dica:* Antes de aplicar o método da bissecção, é necessário encaixotar um zero. Para isto, utilize a análise gráfica ou a busca incremental. Os procedimentos correspondentes estão na última seção deste caderno.

(a) $ x \ln x = 1 $.

(b) $ \cos x = x^2 $.

(c) $ x^5 - 3x^4 - 6x^3 + 4x^2 + 5x - 3 = 0 $.

(d) $ \tan x = x + 2 e^x $.

(e) $ \cos x \cosh x = 1 $, onde por definição $ \cosh x = \frac{e^x + e^{-x}}{2} $.

*Solução:*

## $ \S 7 $ Procedimentos auxiliares<a name="auxiliar"></a>

In [ ]:
def busca_incremental(f, a, b, h):
    """
    Começando com x_1 = a e x_2 = a + h e com incrementos
    de h, retorna o primeiro par de pontos consecutivos onde
    f assume sinais opostos.
    """
    from numpy import sign
    
    # Inicializando:
    x_0 = a
    x_1 = a + h
    f_0 = f(x_0)
    f_1 = f(x_1)
    
    while sign(f_0) == sign(f_1):
        if x_1 > b:
            return None, None
        x_0, f_0 = x_1, f_1
        x_1 += h
        f_1 = f(x_1)
        
    return x_0, x_1

In [ ]:
def plota_funcoes(a, b, N, *fs):
    """
    Entradas:
        * Extremidades a < b de um intervalo.
        * Número N de pontos na amostra dos valores de x.
        * Um número qualquer (>= 1) de funções definidas em [a, b].
    Exibe num mesmo diagrama o gráfico das funções e retorna None.
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    
    x = np.linspace(a, b, N)                    # Amostra de valores de x
    for i, f in enumerate(fs):
        plt.plot(x, f(x), label=f'função {i}')  # Plotar dados: (x, y, etiqueta).
    plt.xlabel('Eixo-x')
    plt.ylabel('Eixo-y')
    plt.grid(True)
    plt.title("Gráfico simples")
    plt.legend()
    
    return None